In [ ]:
import os.path

import panel as pn

from dms_stan.datasets.trpb import (
    ExponRateExponGrowth,
    ExponRateSigmoidGrowth,
    GammaInvRateExponGrowth,
    GammaInvRateSigmoidGrowth,
    LomaxRateExponGrowth,
    LomaxRateSigmoidGrowth,
)


SOURCE_FILE = "~/GitRepos/DMSStan/raw_data/trpb/3-site_merged_replicates/LibI/20230926/LibI_merged_AAs.csv"

# Define all models to be used as well as an output directory for each
MODEL_LIST = [
    (model.from_data_file(SOURCE_FILE), outdir)
    for model, outdir in (
        (LomaxRateExponGrowth, "lomax_expon"),
        (LomaxRateSigmoidGrowth, "lomax_sigmoid"),
        (ExponRateExponGrowth, "expon_expon"),
        (ExponRateSigmoidGrowth, "expon_sigmoid"),
        (GammaInvRateExponGrowth, "gamma_expon"),
        (GammaInvRateSigmoidGrowth, "gamma_sigmoid"),
    )
]

For each model, determine the MAP and run posterior predictive checks:

In [ ]:
def approximate_map():
    plots = []
    for model, outdir in MODEL_LIST:
        print(f"Approximating MAP for {model.__class__.__name__}...")

        # Approximate the MAP
        map_ = model.approximate_map(
            device=1,
            early_stop=10,
            seed=1025,
        )

        # Draw samples from the MAP and save them
        samples = map_.get_inference_obj(seed=1025, batch_size=50)
        samples.save_netcdf(os.path.join(outdir, "approximate_map.nc"))

        # Get and display the posterior predictive checks
        plots.append(samples.run_ppc(logy_ppc_samples=True))

    return plots

# approximate_map()

Approximating MAP for LomaxRateExponGrowth...


Epochs:  15%|█▍        | 14590/100000 [03:35<21:03, 67.59it/s, -log pdf/pmf=1680994.66]


Approximating MAP for LomaxRateSigmoidGrowth...


Epochs:  15%|█▍        | 14865/100000 [03:49<21:53, 64.82it/s, -log pdf/pmf=1681023.76]


Approximating MAP for ExponRateExponGrowth...


...

Approximating MAP for GammaInvRateExponGrowth...


Epochs:  30%|██▉       | 29808/100000 [06:42<15:48, 74.00it/s, -log pdf/pmf=1824501.91]


Approximating MAP for GammaInvRateSigmoidGrowth...


Epochs:  16%|█▌        | 16211/100000 [03:53<20:07, 69.38it/s, -log pdf/pmf=741454.93] 


BokehModel(combine_events=True, render_bundle={'docs_json': {'ea88ca19-03d6-461e-8a4a-115b3f89213e': {'version…

Compile all models to Stan code:

In [ ]:
def compile():
    for model, outdir in MODEL_LIST:
        model.mcmc(
            output_dir=outdir,
            model_name="libI",
            seed=1025,
            delay_run=True,
            iter_warmup=1000,
            force_compile=True,
        )

# compile()

13:47:35 - cmdstanpy - INFO - compiling stan file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/lomax_expon/libI.stan to exe file /home/bwittmann/GitRepos/DMSStan/flip3/trpB/lomax_expon/libI
13:47:54 - cmdstanpy - INFO - compiled model executable: /home/bwittmann/GitRepos/DMSStan/flip3/trpB/lomax_expon/libI
13:47:54 - cmdstanpy - WARNING - Stan compiler has produced 4 warnings:
13:47:54 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=libI.stan --warn-pedantic --O1 --include-paths=/home/bwittmann/GitRepos/DMSStan/dms_stan/model/stan --o=/home/bwittmann/GitRepos/DMSStan/flip3/trpB/lomax_expon/libI.hpp /home/bwittmann/GitRepos/DMSStan/flip3/trpB/lomax_expon/libI.stan
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    no prior is provided, or the prior(s) depend on data variab

Let's take a look at the 